# Multiple analysis between "x categorical" and "y Continuous variable"
### Multiple features vs target

##### [feature 1 & feature 2 vs target]
##### [feature 1 & feature 2 & feature 3 vs target]
- tables
- plots

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

### 0. Package and load data

In [ ]:
from sklearn.datasets import fetch_california_housing
import json
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import combinations

# plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

#### load data categorical features and target generated by percentiles
This is generated in the notebook 0

In [ ]:
data_percentile_feature_target = pd.read_pickle('data/df_cat_feature_target.pkl')
data_percentile_feature_target.head()

#### load data categorical only features generated by percentiles
This is generated in the notebook 0

In [ ]:
data_percentile_feature = pd.read_pickle('data/df_cat_feature.pkl')
data_percentile_feature.head()

#### Define data to use in the example, list features, target
In this notebooks only matters the features, so any of the two datasets can be used

In [ ]:
### data to used
data = data_percentile_feature.copy()

### target
target = 'Price'

### list_features
list_features = list(set(data.columns.tolist()) - set([target]))

## ANALYSIS I. feature1 & feature 2 vs target
Analysis bivariate 2 features categorical vs target

### 1. Codes to eda

In [ ]:
# cross tab feature_x & feature_y with values target
# I want to know the behavior of the target (continuos) between 2 differents features categorical

feature_index_ct = 'quartile_MedInc'
feature_column_ct = 'quartile_HouseAge'
feature_values_ct = 'Price'  # target is a continous variable

ct_mean = pd.crosstab(index = data_percentile_feature[feature_index_ct], 
                      columns = data_percentile_feature[feature_column_ct], 
                      values = data_percentile_feature[feature_values_ct], aggfunc='mean')

ct_mean

In [ ]:
# cross tab feature_x & feature_y with values target. Doble agg function - OPTION 1
feature_index_ct = 'quartile_MedInc'
feature_column_ct = 'quartile_HouseAge'
feature_values_ct = 'Price'  # target is a continous variable

ct_mean_std = pd.crosstab(index = data_percentile_feature[feature_index_ct], 
                      columns = data_percentile_feature[feature_column_ct], 
                      values = data_percentile_feature[feature_values_ct], 
                          aggfunc = ['mean', 'std'])

ct_mean_std

In [ ]:
#### ERROR

# cross tab feature_x & feature_y with values target. Doble agg function - OPTION 2 (two features in the columns)
feature_index_ct = 'quartile_MedInc'
feature_column_ct = 'quartile_HouseAge'
feature_values_ct = 'Price'  # target is a continous variable

# generate a cross tab where the target contiuos is a row and calculate the freq of contiuos target vs pair of features in percentile
ct_freq_target_each_percentile = pd.crosstab(index = data_percentile_feature[feature_values_ct], 
                                             columns = [data_percentile_feature[feature_column_ct], data_percentile_feature[feature_index_ct]])

# aggregation mean std
ct_mean_std = ct_freq_target_each_percentile.agg(['mean', 'std'])
ct_mean_std

In [ ]:
##### ERROR

# cross tab feature_x & feature_y with values target. Doble agg function - OPTION 2 (two features in the rows)
feature_index_ct = 'quartile_MedInc'
feature_column_ct = 'quartile_HouseAge'
feature_values_ct = 'Price'  # target is a continous variable

# generate a cross tab where the target contiuos is the column and calculate the freq of contiuos target vs pair of features in percentile
ct_freq_target_each_percentile = pd.crosstab(
    index = [data_percentile_feature[feature_column_ct], data_percentile_feature[feature_index_ct]],
    columns = data_percentile_feature[feature_values_ct]
                                            )

# aggregation mean std
ct_mean_std = ct_freq_target_each_percentile.agg(['mean', 'std'])

# multiple column (target values) with freq in each cells
ct_mean_std = ct_mean_std.mul(ct_mean_std.columns, axis=1)
ct_mean_std

#### final discovery

In [ ]:
# cross tab feature_x & feature_y with values target. Doble agg function - INDEPEDIENT CROSS TABLES
feature_index_ct = 'quartile_MedInc'
feature_column_ct = 'quartile_HouseAge'
feature_values_ct = 'Price'  # target is a continous variable

ct_mean_mean = pd.crosstab(index = data_percentile_feature[feature_index_ct], 
                      columns = data_percentile_feature[feature_column_ct], 
                      values = data_percentile_feature[feature_values_ct], 
                          aggfunc = 'mean') # agg fuction without list

ct_mean_mean

#### TODO: 
It can change:
- pair of features used in the analysis - so it needs a for between the features
- also change the analysis - mean, std, min, max, custom agregation, etc.

SO the ways to do this, are:
- subplots for pair of feature calculating one aggregation and save in one file. And other aggregations save in other file
- or subplots for aggregation and save differents files with differents pair of features

Choice:
- first one, subplots for pair of feature

### 2. synthesize analysis - plot a subplots of heatmaps of crosstab between 2 features and one aggregation into target

#### auxiliar function - plot heatmap given a input df cross table in format to plot

In [ ]:
def plot_heatmap_crosstable_individual(df_ct, name_table):
    """
    Plot heatmap using the input dataframe with the cross table to plot

    Args
        df_corr (dataframe): dataframe with crosstable to plot

    Return
        fig (figure plotly): fig of plotly with the plot generated
    """
    
    # heatmap
    fig = px.imshow(df_ct, text_auto=True, aspect="auto")
    
    # change title
    fig.update_layout(
      title_text = name_table,
        title_x = 0.5,
    title_font = dict(size = 28)
      )
    
    return fig

#### individual plot

In [ ]:
def crosstab_agg_target_2_features(df, feature_index, feature_column, target, agg_target = 'mean'):
    """
    generate crosstable of aggregation of the target between 2 features (feature index and feature column)

    Args
        ------
    
    Return
        df_ct (dataframe): dataframe with the cross table
    """
    # calculate cross_table
    df_ct= pd.crosstab(index = df[feature_index], 
                          columns = df[feature_column], 
                          values = df[target], 
                              aggfunc = agg_target) # agg fuction without list
    
    # round 3 decimals
    df_ct = df_ct.round(3)
    
    return df_ct

In [ ]:
# params
feature_x_ct = 'quartile_MedInc'
feature_y_ct = 'quartile_HouseAge'
target_ct = 'Price'
agg_target_ct = 'std'

# cross table
individual_crosstab_agg_target_2_features = crosstab_agg_target_2_features(df = data_percentile_feature, 
                                                                                              feature_index = feature_x_ct, 
                                                                                              feature_column = feature_y_ct, 
                                                                                              target = target_ct, 
                                                                                              agg_target = agg_target_ct)

# heatmap
individual_heatmap_agg_target_2_features = plot_heatmap_crosstable_individual(individual_crosstab_agg_target_2_features,
                                        name_table = f'"{agg_target_ct}" of target "{target_ct}" --comparing "{feature_x_ct}" vs "{feature_y_ct}"')

# show
individual_heatmap_agg_target_2_features.show()

### subplot
Use the codes to generate multiple heatmap

Description
function to generate heatmap:
    - generate dataframe crosstab (use the individual function to calculate individual crosstab)
    - plot in plotly the crosstab

In [ ]:
# auxiliar function to map features in a pair (feature_x, feature_y)
def list_map_combinations_features(list_features, dim_combinations = 2):
    """
    Given a list of features of a dataframe, map all the combinations between each features. combinations without replace and (a,b) is the same (b,a)
    IN PREVIOUS CODES THERE ARE OTHER WAY TO MAP THE FEATURESS, ACUALLY THIS WAY IS BETTER

    Args:
        list_features (list): list of features that will generate the combinations
        dim_combinations (string): dimensions of combinations. default 2 -> generate a pair of features (feature_x, feature_y)

    Return
        list_tuple_combinations (list): list where each element is a tuple with the combination
    """
    # get all the possible combinations withtout repeteat
    todas_combinaciones = combinations(list_features, dim_combinations)
    
    # generate output
    list_tuple_combinations = []
    for comb in todas_combinaciones:
        list_tuple_combinations.append(comb)

    return list_tuple_combinations

In [ ]:
def heatmap_crosstab_aggregation_target_2_features(df, target, agg_target = 'mean', number_columns = 1):
    """
    Given a dataframe with columns features + target. Genereate a heatmap of relations between 2 features and one aggregation function of the target
    Detail: 
        Given a dataframe with features categorical, generate a crosstab of aggregation of target between 2 features and plot it in a heatmap
        Calling a individual function to generate a cross table
    
    Args
        df (dataframe): input dataframe with columns features and target
        target (string): target of the dataframe, column that will be delete to plot the relations between only features
        agg_target (string): aggregation function of the target
        number_columns (integer): number of columns. because heatmap could be bigger, plot it into 1 columns by default

    Return
        fig (figure plotly): fig of plotly with the plot generated
    """

    ################# generate a list of tuples of each pair of features to generate the cross table  #####################
    df_only_features = df.drop(columns = target) # delete target of the data
    list_pair_features = list_map_combinations_features(df_only_features.columns.tolist(), 2)

    
    ####################### plot #################################
    
    # calculate number of rows (considering the number of colums passed as args)
    if (len(list_pair_features) % number_columns) != 0:
        number_rows = (len(list_pair_features) // number_columns) + 1
    else:
        number_rows = (len(list_pair_features) // number_columns)

    # create fig to plot
    fig = make_subplots(rows=number_rows, cols=number_columns, 
                        subplot_titles = tuple([str(tupla) for tupla in list_pair_features]) ### title of each subplots
                       )

    ########## for each tuple of features to plot:
    for index_feature, (feature_x, feature_y) in enumerate(list_pair_features):
        
        # get indexes in the subplot (in plotly the indexes starts in 1)
        row = (index_feature // number_columns) + 1
        column = (index_feature % number_columns) + 1

        
        ## get cross table agg function of target, between 2 features - call the INDIVIDUAL FUNCTION TO GENERATE CROSS TABLE 
        ct_agg_target = crosstab_agg_target_2_features(df = df, 
                                                       feature_index = feature_x, 
                                                       feature_column = feature_y, 
                                                       target = target, 
                                                       agg_target = agg_target)
        
        ## tranform cross table freq between pair of features into a heatmap
        fig_aux = px.imshow(ct_agg_target, text_auto=True, aspect="auto")
        
        # add heatmap to fig global
        fig.add_trace(fig_aux.data[0],
            row = row,
            col = column
        )

    # adjust the shape
    fig.update_layout(
        height = 350 * number_rows,  # largo
        width = 850 * number_columns,  # ancho
        title_text =  f'Cross table betweeen "pair of features" with "{agg_target} of the {target}"',
        title_x=0.5,
        title_font = dict(size = 20)
    )

    return fig

In [ ]:
##### MEAN AGG TARGET
fig_crosstab_mean_target_2_features = heatmap_crosstab_aggregation_target_2_features(df = data_percentile_feature, 
                                                                                     target = target, 
                                                                                     agg_target = 'mean', 
                                                                                     number_columns = 1)

fig_crosstab_mean_target_2_features.write_html("output_eda/5_categorical_analysis/crosstab_mean_target_2_features.html")

fig_crosstab_mean_target_2_features.show()

In [ ]:
##### STD AGG TARGET
fig_crosstab_std_target_2_features = heatmap_crosstab_aggregation_target_2_features(df = data_percentile_feature, 
                                                                                     target = target, 
                                                                                     agg_target = 'std', 
                                                                                     number_columns = 1)

fig_crosstab_std_target_2_features.write_html("output_eda/5_categorical_analysis/crosstab_std_target_2_features.html")

fig_crosstab_std_target_2_features.show()

## ANALYSIS II. feature1 & feature 2 & feature 3 vs target
Analysis trivariate 3 features categorical vs target

### individual plot

In [ ]:
def crosstab_agg_target_3_features(df, feature_index1, feature_index2, feature_column, target, agg_target = 'mean'):
    """
    generate crosstable of aggregation of the target between 3 features (2 features indexes and 1 feature column)

    Args
        ------
    
    Return
        ct_2index (dataframe): dataframe with the cross table with multiindex of 2 features
        ct_2index_reset_index (dataframe): dataframe previous with reset of multiindex
    """
    
    ###### calculate cross table 2 index
    ct_2index = pd.crosstab(index = [df[feature_index1], df[feature_index2]], 
                          columns = df[feature_column], 
                          values = df[target], 
                            aggfunc = agg_target)

    
    ##### transform into plotly heatmap format - only one index
    # generate a dataframe with multiindex into only one index (join intro string the multi index into one index)
    
    # 0. reset index
    ct_2index_reset_index = ct_2index.reset_index()
    
    # 1. transform each categorical column into string.
    ct_2index_reset_index[feature_index1] = ct_2index_reset_index[feature_index1].astype(str)
    ct_2index_reset_index[feature_index2] = ct_2index_reset_index[feature_index2].astype(str)
    
    # 2. add name of the column (beacase actually only show q1, q2, etc)
    ct_2index_reset_index[feature_index1] = feature_index1 + '|' + ct_2index_reset_index[feature_index1]
    ct_2index_reset_index[feature_index2] = feature_index2 + '|' + ct_2index_reset_index[feature_index2]
    
    # 3. combine content of 2 columnas and delete old ones
    ct_2index_reset_index['index'] = ct_2index_reset_index[feature_index1] + '__&&__' + ct_2index_reset_index[feature_index2]
    ct_2index_reset_index.drop(columns = [feature_index1, feature_index2], inplace = True)
    ct_2index_reset_index.set_index('index', inplace = True)

    ##### round 3 decimals
    ct_2index = ct_2index.round(3)
    ct_2index_reset_index = ct_2index_reset_index.round(3)
    
    return ct_2index, ct_2index_reset_index

In [ ]:
# params
feature_index1_ct = 'quartile_MedInc'
feature_index2_ct = 'quartile_HouseAge'
feature_column_ct = 'quartile_Population'
target_ct = 'Price'  # target is a continous variable
agg_target_ct = 'mean'

# cross table
_, individual_crosstab_agg_target_3_features = crosstab_agg_target_3_features(df = data_percentile_feature, 
                                                                                                 feature_index1= feature_index1_ct, 
                                                                                                 feature_index2 = feature_index2_ct, 
                                                                                                 feature_column = feature_column_ct, 
                                                                                                 target = target_ct, 
                                                                                                 agg_target = agg_target_ct
                                                                                                )

# heatmap
individual_heatmap_agg_target_3_features = plot_heatmap_crosstable_individual(individual_crosstab_agg_target_3_features,
                    name_table = f'"{agg_target_ct}" of target "{target_ct}" --comparing "{feature_index1_ct}//{feature_index2_ct}" vs "{feature_column_ct}"')

# show
individual_heatmap_agg_target_3_features.show()

# save
individual_heatmap_agg_target_3_features.write_html("output_eda/5_categorical_analysis/heatmap_agg_target_3_features.html")

### subplots
IN THIS CASE IS A SMALL MODIFICATION OF THE FUNCTION THAT PLOT 2 FEATURES

In [ ]:
def heatmap_crosstab_aggregation_target_3_features(df, target, agg_target = 'mean', number_columns = 1):
    """
    Given a dataframe with columns features + target. Genereate a heatmap of relations between 3 features and one aggregation function of the target
    Detail: 
        Given a dataframe with features categorical, generate a crosstab of aggregation of target between 3 features and plot it in a heatmap
        Calling a individual function to generate a cross table
    
    Args
        df (dataframe): input dataframe with columns features and target
        target (string): target of the dataframe, column that will be delete to plot the relations between only features
        agg_target (string): aggregation function of the target
        number_columns (integer): number of columns. because heatmap could be bigger, plot it into 1 columns by default

    Return
        fig (figure plotly): fig of plotly with the plot generated
    """

    ################# generate a list of tuples of each pair of features to generate the cross table  #####################
    df_only_features = df.drop(columns = target) # delete target of the data
    list_triple_features = list_map_combinations_features(df_only_features.columns.tolist(), 3)

    
    ####################### plot #################################
    
    # calculate number of rows (considering the number of colums passed as args)
    if (len(list_triple_features) % number_columns) != 0:
        number_rows = (len(list_triple_features) // number_columns) + 1
    else:
        number_rows = (len(list_triple_features) // number_columns)

    # create fig to plot
    fig = make_subplots(rows=number_rows, cols=number_columns, 
                        subplot_titles = tuple([str(tupla) for tupla in list_triple_features]) ### title of each subplots
                       )

    ########## for each tuple of features to plot:
    for index_feature, (feature_index1_ct, feature_index2_ct, feature_column_ct) in enumerate(list_triple_features):
        
        # get indexes in the subplot (in plotly the indexes starts in 1)
        row = (index_feature // number_columns) + 1
        column = (index_feature % number_columns) + 1

        
        ## get cross table agg function of target, between 3 features - call the INDIVIDUAL FUNCTION TO GENERATE CROSS TABLE 
        _, ct_agg_target_plotly = crosstab_agg_target_3_features(df = df, 
                               feature_index1= feature_index1_ct, 
                               feature_index2 = feature_index2_ct, 
                               feature_column = feature_column_ct, 
                               target = target, 
                               agg_target = agg_target
                              )
        
        ## tranform cross table freq between pair of features into a heatmap
        fig_aux = px.imshow(ct_agg_target_plotly, text_auto=True, aspect="auto")
        
        # add heatmap to fig global
        fig.add_trace(fig_aux.data[0],
            row = row,
            col = column
        )

    # adjust the shape
    fig.update_layout(
        height = 450 * number_rows,  # largo
        width = 1850 * number_columns,  # ancho
        title_text =  f'Cross table betweeen "pair of features" with "{agg_target} of the {target}"',
        title_x=0.5,
        title_font = dict(size = 20)
    )

    return fig

In [ ]:
##### MEAN AGG TARGET
fig_crosstab_mean_target_3_features = heatmap_crosstab_aggregation_target_3_features(df = data_percentile_feature, 
                                                                                     target = target, 
                                                                                     agg_target = 'mean', 
                                                                                     number_columns = 1)

fig_crosstab_mean_target_3_features.write_html("output_eda/5_categorical_analysis/crosstab_mean_target_3_features.html")

#fig_crosstab_mean_target_3_features.show()